Выбранный Датасет: Прогнозирование отказов спутников (Synthetic Satellite Failure Data) - создаем полностью синтетический датасет с характеристиками, имитирующими космические данные, используя make_regression.

Выбранные 5 Регрессоров
Регрессор CatBoost (CatBoostRegressor)
Регрессор LGBM (LGBMRegressor)
Регрессор Экстремальных Деревьев (Extra Trees) (ExtraTreesRegressor)
Регрессия Лассо (Lasso)
Регрессор Дерева Решений (DecisionTreeRegressor)

In [4]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# --- Импорт Регрессоров ---
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso
# Для CatBoost и LGBM
try:
    from lightgbm import LGBMRegressor
    from catboost import CatBoostRegressor
    CATBOOST_PARAMS = {'verbose': 0, 'allow_writing_files': False, 'random_state': 42}
    LGBM_PARAMS = {'random_state': 42}
except ImportError:
    print("Внимание: Библиотеки 'lightgbm' или 'catboost' не установлены. Используются запасные модели (Ridge/Huber).")
    LGBMRegressor = None
    CatBoostRegressor = None
    from sklearn.linear_model import Ridge, HuberRegressor
    LGBMRegressor_alt = Ridge
    CatBoostRegressor_alt = HuberRegressor
    CATBOOST_PARAMS = {} # HuberRegressor does not have random_state
    LGBM_PARAMS = {'random_state': 42} # Ridge does have random_state

# --- 1. Создание Исходного Датасета (Имитация Космических Данных) ---
N_FEATURES = 10 # Характеристики: высота орбиты, мощность, стабильность и т.д.
N_SAMPLES = 500 # Исходное количество спутников

X_orig, y_orig = make_regression(
    n_samples=N_SAMPLES,
    n_features=N_FEATURES,
    n_informative=8,
    noise=50, # Добавим шум, как в реальных данных
    random_state=42,
    shuffle=True
)

feature_names = [f'Feature_{i}' for i in range(N_FEATURES)]
X_orig_df = pd.DataFrame(X_orig, columns=feature_names)
y_orig_series = pd.Series(y_orig, name='Life_Expectancy_Days')

print(f"Исходный размер датасета: {X_orig_df.shape}")

# --- 2. Дополнение Синтетическими Данными (make_regression) ---
# Добавляем 100 синтетических точек (спутников)
N_SYNTH_SAMPLES = 100
X_synth, y_synth = make_regression(
    n_samples=N_SYNTH_SAMPLES,
    n_features=N_FEATURES,
    n_informative=8,
    noise=50,
    random_state=100
)

X_synth_df = pd.DataFrame(X_synth, columns=feature_names)
y_synth_series = pd.Series(y_synth, name='Life_Expectancy_Days')

# Объединение
X_combined = pd.concat([X_orig_df, X_synth_df], ignore_index=True)
y_combined = pd.concat([y_orig_series, y_synth_series], ignore_index=True)

print(f"Размер датасета после добавления синтетики: {X_combined.shape}")

# --- 3. Разделение и Масштабирование Данных ---
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y_combined, test_size=0.2, random_state=42
)

# Масштабирование признаков (обязательно для Lasso)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# --- 4. Обучение и Оценка Моделей ---
models = {
    "CatBoost": CatBoostRegressor(**CATBOOST_PARAMS) if CatBoostRegressor else CatBoostRegressor_alt(**CATBOOST_PARAMS),
    "LGBM": LGBMRegressor(**LGBM_PARAMS) if LGBMRegressor else LGBMRegressor_alt(**LGBM_PARAMS),
    "Extra Trees": ExtraTreesRegressor(random_state=42),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Lasso Regression": Lasso(alpha=0.1, random_state=42), # Alpha для регуляризации
}

results = []

for name, model in models.items():
    if name == "Lasso Regression":
        # Линейные модели обучаем на масштабированных данных
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        # Ансамбли и деревья обучаем на немасштабированных данных
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    results.append({
        "Модель": name,
        "RMSE (меньше - лучше)": round(rmse, 4),
        "R2 (ближе к 1 - лучше)": round(r2, 4)
    })

# --- 5. Вывод Результатов ---
results_df = pd.DataFrame(results).sort_values(by="RMSE (меньше - лучше)")
print("\n" + "="*70)
print("Сравнение Регрессионных Моделей (Прогнозирование Срока Службы Спутников)")
print("="*70)
print(results_df.to_markdown(index=False))

Внимание: Библиотеки 'lightgbm' или 'catboost' не установлены. Используются запасные модели (Ridge/Huber).
Исходный размер датасета: (500, 10)
Размер датасета после добавления синтетики: (600, 10)

Сравнение Регрессионных Моделей (Прогнозирование Срока Службы Спутников)
| Модель           |   RMSE (меньше - лучше) |   R2 (ближе к 1 - лучше) |
|:-----------------|------------------------:|-------------------------:|
| Lasso Regression |                 59.6678 |                   0.833  |
| LGBM             |                 59.6992 |                   0.8329 |
| CatBoost         |                 59.8486 |                   0.832  |
| Extra Trees      |                 72.4077 |                   0.7541 |
| Decision Tree    |                111.934  |                   0.4124 |
